In [77]:
%load_ext autoreload

%autoreload 
from getdata import YahooFinance
from getdata import SQLrepo
import sqlite3
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from statsmodels.tsa.arima.model import ARIMA
from statsmodels.tsa.ar_model import AutoReg
from dateutil.parser import parse


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [5]:
db_name = "stocks.sqlite"
connection=sqlite3.connect(database=db_name,check_same_thread=False)
repo=SQLrepo(connection)

In [100]:
df1=repo.read_table(table_name="Google")
df1.head(10)

,Date,Close
0,2004-08-19 00:00:00-04:00,2.499133
1,2004-08-20 00:00:00-04:00,2.697639
2,2004-08-23 00:00:00-04:00,2.724787
3,2004-08-24 00:00:00-04:00,2.611960
4,2004-08-25 00:00:00-04:00,2.640104
5,2004-08-26 00:00:00-04:00,2.687676
6,2004-08-27 00:00:00-04:00,2.643840
7,2004-08-30 00:00:00-04:00,2.540727
8,2004-08-31 00:00:00-04:00,2.549693
9,2004-09-01 00:00:00-04:00,2.496891


In [101]:
df=repo.read_table(table_name="Google")

df['Date'] = pd.to_datetime(df['Date'].str.replace(r'-\d{2}:\d{2}$', '', regex=True))
df.set_index('Date', inplace=True)

df=df.resample('d').ffill().squeeze()


In [102]:
df.head(10)

Date
2004-08-19    2.499133
2004-08-20    2.697639
2004-08-21    2.697639
2004-08-22    2.697639
2004-08-23    2.724787
2004-08-24    2.611960
2004-08-25    2.640104
2004-08-26    2.687676
2004-08-27    2.643840
2004-08-28    2.643840
Freq: D, Name: Close, dtype: float64

In [59]:
cut=int(len(df) * 0.80)
y_train= df.iloc[:cut]
y_test = df.iloc[cut:]
y_test.head(15)

Date
2014-11-18 00:00:00+00:00    25.577393
2014-11-19 00:00:00+00:00    25.909472
2014-11-20 00:00:00+00:00    25.729969
2014-11-21 00:00:00+00:00    26.097960
2014-11-24 00:00:00+00:00    26.133860
2014-11-25 00:00:00+00:00    26.618521
2014-11-26 00:00:00+00:00    26.387407
2014-11-27 00:00:00+00:00    26.701551
2014-11-28 00:00:00+00:00    26.701551
2014-12-01 00:00:00+00:00    26.685835
2014-12-02 00:00:00+00:00    25.819721
2014-12-03 00:00:00+00:00    25.720993
2014-12-04 00:00:00+00:00    26.012697
2014-12-05 00:00:00+00:00    25.913961
2014-12-08 00:00:00+00:00    25.804014
Freq: B, Name: Close, dtype: float64

In [ ]:
returns= df.pct_change().dropna()